# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2022 2:18PM,239787,16,8122624,Sartorius Bioprocess Solutions,Released
1,Jun 16 2022 2:18PM,239787,16,8144166,Sartorius Bioprocess Solutions,Released
2,Jun 16 2022 2:18PM,239787,16,8144219,Sartorius Bioprocess Solutions,Released
3,Jun 16 2022 2:18PM,239787,16,8144237,Sartorius Bioprocess Solutions,Released
4,Jun 16 2022 2:18PM,239787,16,8147993,Sartorius Bioprocess Solutions,Released
5,Jun 16 2022 2:18PM,239787,16,8148563,Sartorius Bioprocess Solutions,Released
6,Jun 16 2022 2:18PM,239787,16,8149225,Sartorius Bioprocess Solutions,Released
7,Jun 16 2022 2:15PM,239784,10,MSP211317,"Methapharm, Inc.",Released
8,Jun 16 2022 2:15PM,239784,10,MSP211318,"Methapharm, Inc.",Released
9,Jun 16 2022 2:15PM,239784,10,MSP211319,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,239781,Released,1
31,239783,Released,3
32,239784,Released,31
33,239786,Released,1
34,239787,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239781,NaN,NaN,1.0
239783,NaN,NaN,3.0
239784,NaN,NaN,31.0
239786,NaN,NaN,1.0
239787,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239691,0.0,0.0,1.0
239692,0.0,0.0,1.0
239694,0.0,0.0,1.0
239696,0.0,0.0,1.0
239699,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239691,0,0,1
239692,0,0,1
239694,0,0,1
239696,0,0,1
239699,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239691,0,0,1
1,239692,0,0,1
2,239694,0,0,1
3,239696,0,0,1
4,239699,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239691,,,1
1,239692,,,1
2,239694,,,1
3,239696,,,1
4,239699,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2022 2:18PM,239787,16,Sartorius Bioprocess Solutions
7,Jun 16 2022 2:15PM,239784,10,"Methapharm, Inc."
38,Jun 16 2022 2:14PM,239786,10,Amatheon Pharmaceuticals
39,Jun 16 2022 2:12PM,239783,10,"Snap Medical Industries, LLC"
42,Jun 16 2022 2:02PM,239781,10,Acute Outpatient Solutions
43,Jun 16 2022 1:59PM,239780,12,Hush Hush
47,Jun 16 2022 1:51PM,239779,10,ISDIN Corporation
61,Jun 16 2022 1:34PM,239777,10,Emerginnova
62,Jun 16 2022 1:26PM,239775,20,Alumier Labs Inc.
77,Jun 16 2022 12:54PM,239772,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 16 2022 2:18PM,239787,16,Sartorius Bioprocess Solutions,,,7
1,Jun 16 2022 2:15PM,239784,10,"Methapharm, Inc.",,,31
2,Jun 16 2022 2:14PM,239786,10,Amatheon Pharmaceuticals,,,1
3,Jun 16 2022 2:12PM,239783,10,"Snap Medical Industries, LLC",,,3
4,Jun 16 2022 2:02PM,239781,10,Acute Outpatient Solutions,,,1
5,Jun 16 2022 1:59PM,239780,12,Hush Hush,,,4
6,Jun 16 2022 1:51PM,239779,10,ISDIN Corporation,,,14
7,Jun 16 2022 1:34PM,239777,10,Emerginnova,,,1
8,Jun 16 2022 1:26PM,239775,20,Alumier Labs Inc.,,,15
9,Jun 16 2022 12:54PM,239772,10,Emerginnova,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 2:18PM,239787,16,Sartorius Bioprocess Solutions,7,,
1,Jun 16 2022 2:15PM,239784,10,"Methapharm, Inc.",31,,
2,Jun 16 2022 2:14PM,239786,10,Amatheon Pharmaceuticals,1,,
3,Jun 16 2022 2:12PM,239783,10,"Snap Medical Industries, LLC",3,,
4,Jun 16 2022 2:02PM,239781,10,Acute Outpatient Solutions,1,,
5,Jun 16 2022 1:59PM,239780,12,Hush Hush,4,,
6,Jun 16 2022 1:51PM,239779,10,ISDIN Corporation,14,,
7,Jun 16 2022 1:34PM,239777,10,Emerginnova,1,,
8,Jun 16 2022 1:26PM,239775,20,Alumier Labs Inc.,15,,
9,Jun 16 2022 12:54PM,239772,10,Emerginnova,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 2:18PM,239787,16,Sartorius Bioprocess Solutions,7,,
1,Jun 16 2022 2:15PM,239784,10,"Methapharm, Inc.",31,,
2,Jun 16 2022 2:14PM,239786,10,Amatheon Pharmaceuticals,1,,
3,Jun 16 2022 2:12PM,239783,10,"Snap Medical Industries, LLC",3,,
4,Jun 16 2022 2:02PM,239781,10,Acute Outpatient Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 2:18PM,239787,16,Sartorius Bioprocess Solutions,7.0,NaN,NaN
1,Jun 16 2022 2:15PM,239784,10,"Methapharm, Inc.",31.0,NaN,NaN
2,Jun 16 2022 2:14PM,239786,10,Amatheon Pharmaceuticals,1.0,NaN,NaN
3,Jun 16 2022 2:12PM,239783,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
4,Jun 16 2022 2:02PM,239781,10,Acute Outpatient Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 2:18PM,239787,16,Sartorius Bioprocess Solutions,7.0,0.0,0.0
1,Jun 16 2022 2:15PM,239784,10,"Methapharm, Inc.",31.0,0.0,0.0
2,Jun 16 2022 2:14PM,239786,10,Amatheon Pharmaceuticals,1.0,0.0,0.0
3,Jun 16 2022 2:12PM,239783,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
4,Jun 16 2022 2:02PM,239781,10,Acute Outpatient Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2877086,67.0,16.0,0.0
12,479529,5.0,0.0,0.0
15,479525,21.0,0.0,0.0
16,1438520,12.0,0.0,0.0
19,239752,1.0,0.0,0.0
20,1438371,25.0,23.0,4.0
22,479449,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2877086,67.0,16.0,0.0
1,12,479529,5.0,0.0,0.0
2,15,479525,21.0,0.0,0.0
3,16,1438520,12.0,0.0,0.0
4,19,239752,1.0,0.0,0.0
5,20,1438371,25.0,23.0,4.0
6,22,479449,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,67.0,16.0,0.0
1,12,5.0,0.0,0.0
2,15,21.0,0.0,0.0
3,16,12.0,0.0,0.0
4,19,1.0,0.0,0.0
5,20,25.0,23.0,4.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,67.0
1,12,Released,5.0
2,15,Released,21.0
3,16,Released,12.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,4.0,0.0
Executing,16.0,0.0,0.0,0.0,0.0,23.0,0.0
Released,67.0,5.0,21.0,12.0,1.0,25.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,16.0,0.0,0.0,0.0,0.0,23.0,0.0
2,Released,67.0,5.0,21.0,12.0,1.0,25.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,16.0,0.0,0.0,0.0,0.0,23.0,0.0
2,Released,67.0,5.0,21.0,12.0,1.0,25.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()